In [3]:
from algobox.client import ApiClient, DataCollectorClient
from algobox.analysis import QuickAnalysis
from datetime import datetime, timedelta
%matplotlib notebook

In [63]:
INSTRUMENT_DAX = 'DE30_EUR'

def print_connections(client):
    connections_client = client.connections_client
    for connection in connections_client.find_all_connections():
        status = connections_client.get_connection_status(connection.connection_id)
        subscriptions = [x.value for x in connections_client.get_subscribed_instruments(connection.connection_id)]
        print("Connection [%s:%s], status [%s], subscriptions %s" % (
                connection.connection_id, connection.connector_id, status.connection_status, subscriptions))


def print_strategies(client):
    strategies_client = client.strategies_client
    for strategy in strategies_client.get_active_strategies():
        status = strategies_client.get_strategy_status(strategy.instance_id)
        print("----")
        print("Strategy [%s:%s]" % (strategy.instance_id, strategy.strategy_id))
        print("State [%s], received ticks [%s]" % (status.status, status.received_ticks))
        print("Mappings:")
        for mapping in strategy.instruments_mapping:            
            print("- price [%s:%s], order [%s:%s]" % (
                    mapping.price_instrument_id, mapping.price_connection_id, mapping.order_instrument_id, mapping.order_connection_id))
        print("Parameters:")
        for key, value in strategy.parameters.items():
            print('- [%s]=[%s]' % (key, value))
        print("Log:")
        for log in strategies_client.get_strategy_events_log(strategy.instance_id, page_number=0, page_size=5):
            print("- %d %s: [%s] (data: %s) " % (log.timestamp, log.strategy_event_type, log.message, log.data))
        print("")
        

def print_strategies_history(client):
    strategies_client = client.strategies_client
    for strategy in strategies_client.get_strategies_history(page_number=0, page_size=5):
        print("----")
        print("Strategy [%s:%s:%s]" % (strategy.instance_id, strategy.strategy_id, strategy.title))
        print("Received ticks [%s]" % strategy.received_ticks)
        if strategy.exception_message:
            print("Exception [%s], stack [%s]" % (strategy.exception_message, strategy.exception_stack))
        print("")

def get_last_trading_day():
    previous_day = datetime.now()
    if previous_day.weekday() == 6:
        return previous_day + timedelta(days=-2)
    else:
        return previous_day + timedelta(days=-1)

In [5]:
api_client = ApiClient()

## Connections

In [6]:
print_connections(api_client)

Connection [oanda-demo:oanda], status [DISCONNECTED], subscriptions ['DE30_EUR']


## Strategies

In [64]:
print_strategies(api_client)

----
Strategy [bdbb56c6-0dba-4a53-8443-a936fbc3a41e:daxLevelsLollipop]
State [UNDEFINED], received ticks [0]
Mappings:
- price [DE30_EUR:oanda-demo], order [DE30_EUR:oanda-demo]
Parameters:
- [validFromMinutes]=[30]
- [timeframe]=[M15]
- [slPips]=[10]
- [minLollipopSize]=[9.0]
- [validToMinutes]=[30]
- [maxLollipopSize]=[30.0]
- [tpPips]=[12]
- [minHeadDistanceFromLevel]=[11.0]
- [maxTailPipsFromLevel]=[1.0]
Log:

----
Strategy [9c5c9222-2901-4e45-8e04-c1c4b4d63c4f:daxLevelsLollipop]
State [UNDEFINED], received ticks [0]
Mappings:
- price [DE30_EUR:oanda-demo], order [DE30_EUR:oanda-demo]
Parameters:
- [validFromMinutes]=[30]
- [timeframe]=[M5]
- [slPips]=[10]
- [minLollipopSize]=[9.0]
- [validToMinutes]=[30]
- [maxLollipopSize]=[30.0]
- [tpPips]=[12]
- [minHeadDistanceFromLevel]=[11.0]
- [maxTailPipsFromLevel]=[1.0]
Log:



In [ ]:
print_strategies_history(api_client)

## Prices

In [ ]:
analysis = QuickAnalysis(algobox_client=api_client)
overview = analysis.day_overview(instrument_id=INSTRUMENT_DAX, date=get_last_trading_day())
if overview.plot is not None:
    overview.plot.show()

# Datacollector

In [ ]:
datacollector_client = DataCollectorClient()

## Connections


In [ ]:
print_connections(datacollector_client)

## Last tick

In [ ]:
prices_stage_client = datacollector_client.pricestages_client
pices_count = prices_stage_client.count_prices(INSTRUMENT_DAX)
last_price = prices_stage_client.find_last_price(INSTRUMENT_DAX)
print('Prices in stage [%d], last price [%r], time [%s].' % (pices_count, last_price, datetime.utcfromtimestamp(last_price.time / 1000)))